In [1]:
import os
%pwd

'/mnt/cb03386d-9344-47b1-82f9-868fbb64b4ae/python_projects/DNA_classifier/research'

In [2]:
os.chdir("../")
%pwd

'/mnt/cb03386d-9344-47b1-82f9-868fbb64b4ae/python_projects/DNA_classifier'

In [3]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path
    ngram: int

In [4]:
from dnaseq.constants import *
from dnaseq.utils.common import read_yaml, create_directories

class ConfigurationManager:
    def __init__(
        self,
        config_file_path = CONFIG_FILE_PATH,
        params_file_path = PARAMS_FILE_PATH,
        schema_file_path = SCHEMA_FILE_PATH
    ):
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)
        self.schema = read_yaml(schema_file_path)
        
        create_directories([self.config.artifacts_root])
        
    def get_data_tranformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation
        params = self.params.count_vectorizer
        
        create_directories([config.root_dir])
        
        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            ngram=params.ngram
        )
        
        return data_transformation_config

In [7]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sentence_transformers import SentenceTransformer
import scipy as sp
from dnaseq import logger

class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config

    def transform_data(self):
        def getKmers(sequence, size=6):
            return [sequence[x:x+size].lower() for x in range(len(sequence) - size + 1)]
        
        data = pd.read_table(self.config.data_path)
        data['kmers'] = data.apply(lambda x: getKmers(x['sequence']), axis=1)
        data = data.drop('sequence', axis=1)
        
        X_data = list(data['kmers'])
        for item in range(len(X_data)):
            X_data[item] = ' '.join(X_data[item])
            
        y_data = data.iloc[:, 0].values
        
        ngram_range = self.config.ngram, self.config.ngram
        cv = CountVectorizer(ngram_range=ngram_range)
        X_data_cv = cv.fit_transform(X_data)
        logger.info("Transformed train data with CoountVectorizer")
        logger.info(X_data_cv.shape)
        logger.info(type(X_data_cv))
        
        model = SentenceTransformer('all-MiniLM-L6-v2')
        X_data_emb = model.encode(X_data)
        logger.info("Transformed train data with SentenceTransformer")
        logger.info(X_data_emb.shape)
        logger.info(type(X_data_emb))
        
        return X_data_cv, X_data_emb, y_data
    
    def train_test_spliting(self, x_cv, x_emb, y):
        X_train_cv, X_test_cv, y_train, y_test = train_test_split(
            x_cv,
            y,
            test_size = 0.20,
            random_state=42
        )
        
        X_train_emb, X_test_emb, y_train, y_test = train_test_split(
            x_emb,
            y,
            test_size = 0.20,
            random_state=42
        )
        
        sp.sparse.save_npz(os.path.join(self.config.root_dir, 'X_train_cv.npz'), X_train_cv)
        sp.sparse.save_npz(os.path.join(self.config.root_dir,'X_test_cv.npz'), X_test_cv)
        pd.DataFrame(X_train_emb).to_csv(os.path.join(self.config.root_dir, "X_train_emb.csv"), index=False)
        pd.DataFrame(X_test_emb).to_csv(os.path.join(self.config.root_dir, "X_test_emb.csv"), index=False)
        pd.DataFrame(y_train).to_csv(os.path.join(self.config.root_dir, "y_train.csv"), index=False)
        pd.DataFrame(y_test).to_csv(os.path.join(self.config.root_dir, "y_test.csv"), index=False)
        
        logger.info("Splited data into training and test sets")

In [8]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_tranformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    x_cv, x_emb, y = data_transformation.transform_data()
    data_transformation.train_test_spliting(x_cv, x_emb, y)

except Exception as e:
    raise e

[2024-03-29 01:42:48,644: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-03-29 01:42:48,658: INFO: common: yaml file: params.yaml loaded successfully]
[2024-03-29 01:42:48,660: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-03-29 01:42:48,662: INFO: common: created directory at: artifacts]
[2024-03-29 01:42:48,663: INFO: common: created directory at: artifacts/data_transformation]
[2024-03-29 01:42:57,559: INFO: 2054095053: Transformed train data with CoountVectorizer]
[2024-03-29 01:42:57,563: INFO: 2054095053: (4380, 602855)]
[2024-03-29 01:42:57,564: INFO: 2054095053: <class 'scipy.sparse._csr.csr_matrix'>]
[2024-03-29 01:42:57,565: INFO: SentenceTransformer: Load pretrained SentenceTransformer: all-MiniLM-L6-v2]
[2024-03-29 01:42:59,648: INFO: SentenceTransformer: Use pytorch device_name: cuda]


Batches:   0%|          | 0/137 [00:00<?, ?it/s]

[2024-03-29 01:43:28,341: INFO: 2054095053: Transformed train data with SentenceTransformer]
[2024-03-29 01:43:28,344: INFO: 2054095053: (4380, 384)]
[2024-03-29 01:43:28,345: INFO: 2054095053: <class 'numpy.ndarray'>]
[2024-03-29 01:43:31,797: INFO: 2054095053: Splited data into training and test sets]
